<a href="https://colab.research.google.com/github/VictorHugoMartins/israel_x_palestine_data_analysis/blob/main/preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Experimentos com Modelos de Toxicidade

In [1]:
!pip install transformers

In [2]:
!pip install detoxify pandas torch

In [3]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121  # Use 'cpu' se não for usar GPU

Found existing installation: torch 2.8.0+cu126
Uninstalling torch-2.8.0+cu126:
  Successfully uninstalled torch-2.8.0+cu126
Found existing installation: torchvision 0.23.0+cu126
Uninstalling torchvision-0.23.0+cu126:
  Successfully uninstalled torchvision-0.23.0+cu126
Found existing installation: torchaudio 2.8.0+cu126
Uninstalling torchaudio-2.8.0+cu126:
  Successfully uninstalled torchaudio-2.8.0+cu126
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 84.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 98.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/

In [4]:
!pip install scikit-learn

In [5]:
!pip install torch

In [6]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

def plot_ground_truth_por_classe(df_merged, tox_score, label_cols):
  df_merged['label_final'] = df_merged[label_cols].mode(axis=1)[0]
  print(df_merged['label_final'].value_counts())

  return df_merged

In [7]:
import pandas as pd

def ler_df(folder_path):
  df_videos = pd.read_csv(folder_path)
  print("temos quantos videos? ", len(df_videos))
  return df_videos

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def tabela_anotacao_por_faixa(df_merged, tox_score, min_val=None, max_val=None, n_bins=5):
    tox_score = tox_score.strip()

    # 1. Resolver colunas duplicadas
    col_mask = [col == tox_score for col in df_merged.columns]
    col_indices = [i for i, x in enumerate(col_mask) if x]

    if len(col_indices) > 1:
        print(f"Coluna duplicada detectada: '{tox_score}' aparece {len(col_indices)} vezes")
        null_counts = [df_merged.iloc[:, i].isnull().sum() for i in col_indices]
        best_index = col_indices[np.argmin(null_counts)]
        for i in reversed(col_indices):
            if i != best_index:
                df_merged.drop(columns=df_merged.columns[i], inplace=True)
        print(f"Coluna '{tox_score}' mantida na posição {best_index} com menos nulos")

    if tox_score not in df_merged.columns:
        df_merged['tox_sexism_score'] = df_merged['tox_sexism_pred']
        tox_score = 'tox_sexism_score'

    # 2. Filtrar textos válidos
    df_valid = df_merged[df_merged[tox_score].notnull()].copy()

    # 3. Ordenar pelo score
    df_valid = df_valid.sort_values(by=tox_score).reset_index(drop=True)

    total_textos = len(df_valid)
    textos_por_faixa = total_textos // n_bins
    total_utilizavel = textos_por_faixa * n_bins
    df_uniforme = df_valid.iloc[:total_utilizavel].copy()

    # 4. Gerar rótulos de intervalo (bins) com base nos scores
    min_score = df_uniforme[tox_score].min() if min_val is None else min_val
    max_score = df_uniforme[tox_score].max() if max_val is None else max_val
    bins = np.linspace(min_score, max_score, n_bins + 1)
    faixa_labels = [f'{round(bins[i], 2)}-{round(bins[i+1], 2)}' for i in range(len(bins)-1)]

    # 5. Atribuir rótulos com base no índice (não pelo valor do score!)
    faixa_indices = np.repeat(faixa_labels, textos_por_faixa)
    df_uniforme["toxicity_range"] = faixa_indices

    # 6. Agrupar por faixa e contar os labels finais
    result = df_uniforme.groupby(['toxicity_range', 'label_final']).size().unstack(fill_value=0)

    print("Anotação baseada na Moda")
    print(result)

    # 7. Plotar as matrizes de confusão por faixa
    if 'label_final' in df_uniforme.columns and 'label_pred' in df_uniforme.columns:
        confusao_long = df_uniforme.groupby(['toxicity_range', 'label_final', 'label_pred']).size().reset_index(name='count')
        confusao_long = confusao_long.rename(columns={'label_final': 'true', 'label_pred': 'pred'})

        faixa_order = sorted(df_uniforme['toxicity_range'].unique(), key=lambda x: float(str(x).split('-')[0]))
        num_faixas = len(faixa_order)

        ncols = 5
        nrows = (num_faixas + ncols - 1) // ncols

        fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(5 * ncols, 4 * nrows), constrained_layout=True)
        axes = axes.flatten()

        for i, faixa in enumerate(faixa_order):
            df_faixa = confusao_long[confusao_long['toxicity_range'] == faixa]
            pivot_df = df_faixa.pivot(index='true', columns='pred', values='count').fillna(0)

            ax = axes[i]
            sns.heatmap(pivot_df, annot=True, fmt='d', cmap='Blues', ax=ax)
            ax.set_title(f'Matriz de Confusão\nFaixa {faixa}')
            ax.set_xlabel('Predito')
            ax.set_ylabel('Real')

        for j in range(num_faixas, len(axes)):
            fig.delaxes(axes[j])

        plt.suptitle("Matriz de Confusão por Faixa", fontsize=18)
        plt.show()

    return result


In [9]:
import seaborn as sns
import pandas as pd
import numpy as np

def plot_all_confusion_matrices_in_grid(df_cm_long, thresholds, title):
    """
    Plota 10 matrizes de confusão em uma única figura em uma grade de 2x5.

    Args:
        df_cm_long (pd.DataFrame): DataFrame com colunas 'threshold', 'true', 'pred' e 'value_abs'.
        thresholds (list or np.array): A lista de thresholds a serem plotados.
        title (str): Título principal do gráfico.
    """
    # Cria a figura e a grade de subplots
    fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(20, 8))
    axes = axes.flatten()  # Acha os eixos para iterar facilmente

    # Itera sobre cada threshold e plota uma matriz em um subplot
    for idx, t in enumerate(thresholds):
        ax = axes[idx]

        # Filtra o DataFrame para o threshold atual
        df_filtered = df_cm_long[df_cm_long['threshold'] == t]

        if df_filtered.empty:
            print(f"Nenhum dado encontrado para o threshold {t}")
            continue

        # Transforma para o formato de matriz
        cm_matrix = df_filtered.pivot(index='true', columns='pred', values='value_abs')

        # Plota o heatmap no subplot atual
        sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='Blues', cbar=False,
                    xticklabels=['Previsto 0', 'Previsto 1'],
                    yticklabels=['Real 0', 'Real 1'], ax=ax)

        ax.set_title(f'Threshold: {t:.1f}')
        ax.set_xlabel('Classe Prevista')
        ax.set_ylabel('Classe Real')

    # Configura o título principal e o layout
    fig.suptitle(title, fontsize=16)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Ajusta o layout para evitar sobreposição do título
    plt.show()

In [33]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def threshold_experiment_and_classify(
    df,
    label_cols,
    tox_score='toxicity',
    do_plots=True
):
    # Garantir numérico
    df[tox_score] = pd.to_numeric(df[tox_score], errors='coerce')

    thresholds = np.linspace(0.1, 1.0, 10)

    # Listas para as métricas macro (média entre classes)
    f1s_macro, precisions_macro, recalls_macro = [], [], []

    # Listas para as métricas por classe
    f1s_per_class = {'0': [], '1': []}
    prec_per_class = {'0': [], '1': []}
    rec_per_class = {'0': [], '1': []}

    # Matrizes de confusão por threshold (absolutas e normalizadas)
    conf_matrices_abs = []
    conf_matrices_norm = []

    # 1) Label final por maioria entre anotadores
    df['label_final'] = df[label_cols].mode(axis=1)[0].astype(int)

    # 2) Varre thresholds
    for t in thresholds:
        df['predicted'] = (df[tox_score] > t).astype(int)

        # average=None -> por classe; labels=[0,1] para ordem estável
        f1 = f1_score(df['label_final'], df['predicted'], average=None, labels=[0,1], zero_division=0)
        pre = precision_score(df['label_final'], df['predicted'], average=None, labels=[0,1], zero_division=0)
        rec = recall_score(df['label_final'], df['predicted'], average=None, labels=[0,1], zero_division=0)

        # Guarda macro (média simples entre classes)
        f1s_macro.append(np.mean(f1))
        precisions_macro.append(np.mean(pre))
        recalls_macro.append(np.mean(rec))

        # Guarda por classe
        f1s_per_class['0'].append(float(f1[0]));  f1s_per_class['1'].append(float(f1[1]))
        prec_per_class['0'].append(float(pre[0])); prec_per_class['1'].append(float(pre[1]))
        rec_per_class['0'].append(float(rec[0]));  rec_per_class['1'].append(float(rec[1]))

        # Matrizes de confusão
        cm_abs = confusion_matrix(df['label_final'], df['predicted'], labels=[0,1])
        cm_norm = confusion_matrix(df['label_final'], df['predicted'], labels=[0,1], normalize='true')
        conf_matrices_abs.append(cm_abs)
        conf_matrices_norm.append(cm_norm)

    # ======= DataFrames requisitados =======
    # (1) F1 por classe vs threshold (wide)
    df_f1_por_classe = pd.DataFrame({
        'threshold': thresholds,
        'f1_class_0': f1s_per_class['0'],
        'f1_class_1': f1s_per_class['1']
    })

    # (2) Precision por classe vs threshold (wide)
    df_precision_por_classe = pd.DataFrame({
        'threshold': thresholds,
        'precision_class_0': prec_per_class['0'],
        'precision_class_1': prec_per_class['1']
    })

    # (3) Recall por classe vs threshold (wide)
    df_recall_por_classe = pd.DataFrame({
        'threshold': thresholds,
        'recall_class_0': rec_per_class['0'],
        'recall_class_1': rec_per_class['1']
    })

    # (4) Matrizes de confusão em formato longo
    def _cm_list_to_long_df(cm_list, thresholds, value_col):
        rows = []
        for idx, t in enumerate(thresholds):
            cm = cm_list[idx]
            for i, true_label in enumerate([0, 1]):
                for j, pred_label in enumerate([0, 1]):
                    rows.append({
                        'threshold': float(t),
                        'true': int(true_label),
                        'pred': int(pred_label),
                        value_col: float(cm[i, j]) if value_col != 'value_abs' else int(cm[i, j])
                    })
        return pd.DataFrame(rows)

    df_cm_abs_long = _cm_list_to_long_df(conf_matrices_abs, thresholds, value_col='value_abs')
    df_cm_norm_long = _cm_list_to_long_df(conf_matrices_norm, thresholds, value_col='value_norm')

    # (5) Melhor threshold pelo F1 macro
    best_index = int(np.argmax(f1s_macro))
    best_theta = float(thresholds[best_index])

    # (6) Classificação final com melhor threshold
    df['toxicity_classification'] = (df[tox_score] > best_theta).astype(int)

    # Adicionar a contagem por faixa de toxicidade
    bins = np.linspace(0.0, 1.0, 11)
    labels = [f'{x:.1f}-{y:.1f}' for x, y in zip(bins[:-1], bins[1:])]
    df['toxicity_range'] = pd.cut(df[tox_score], bins=bins, labels=labels, right=False, include_lowest=True)
    counts_df = df.groupby(['toxicity_range', 'toxicity_classification']).size().unstack(fill_value=0)
    counts_df.columns = [f'predicted_{col}' for col in counts_df.columns]

    # (7) Métricas finais
    final_precision = precision_score(df['label_final'], df['toxicity_classification'],
                                      average=None, labels=[0,1], zero_division=0)
    final_recall = recall_score(df['label_final'], df['toxicity_classification'],
                                     average=None, labels=[0,1], zero_division=0)
    final_f1 = f1_score(df['label_final'], df['toxicity_classification'],
                                average=None, labels=[0,1], zero_division=0)
    final_accuracy = accuracy_score(df['label_final'], df['toxicity_classification'])

    # Suportes por classe
    supports = df['label_final'].value_counts().reindex([0,1]).fillna(0).astype(int).to_dict()

    # DataFrame com métricas finais por classe
    df_final_por_classe = pd.DataFrame([
        {
            'classe': 0,
            'support': int(supports.get(0, 0)),
            'precision': float(final_precision[0]),
            'recall': float(final_recall[0]),
            'f1': float(final_f1[0]),
        },
        {
            'classe': 1,
            'support': int(supports.get(1, 0)),
            'precision': float(final_precision[1]),
            'recall': float(final_recall[1]),
            'f1': float(final_f1[1]),
        }
    ])

    metrics = {
        'best_threshold': best_theta,
        'thresholds': thresholds.tolist(),
        'threshold_curves': {
            'macro': {
                'f1': f1s_macro,
                'precision': precisions_macro,
                'recall': recalls_macro
            },
            'per_class': {
                '0': {
                    'f1': f1s_per_class['0'],
                    'precision': prec_per_class['0'],
                    'recall': rec_per_class['0']
                },
                '1': {
                    'f1': f1s_per_class['1'],
                    'precision': prec_per_class['1'],
                    'recall': rec_per_class['1']
                }
            },
            'confusion_matrices': {
                'absolute': [cm.tolist() for cm in conf_matrices_abs],
                'normalized': [cm.tolist() for cm in conf_matrices_norm]
            }
        },
        'final': {
            'accuracy': float(final_accuracy),
            'per_class': {
                '0': {
                    'support': int(supports.get(0, 0)),
                    'precision': float(final_precision[0]),
                    'recall': float(final_recall[0]),
                    'f1': float(final_f1[0]),
                },
                '1': {
                    'support': int(supports.get(1, 0)),
                    'precision': float(final_precision[1]),
                    'recall': float(final_recall[1]),
                    'f1': float(final_f1[1]),
                }
            }
        }
    }

    # Limpeza da coluna auxiliar
    df.drop(columns=['predicted'], errors='ignore', inplace=True)

    # Pacote de DataFrames para retornar
    dataframes = {
        'f1_por_classe_wide': df_f1_por_classe,
        'precision_por_classe_wide': df_precision_por_classe,
        'recall_por_classe_wide': df_recall_por_classe,
        'confusao_abs_long': df_cm_abs_long,
        'confusao_norm_long': df_cm_norm_long,
        'final_por_classe': df_final_por_classe
    }

    return df, metrics, dataframes

Mostra todas as métricas do detoxify

In [32]:
import numpy as np

# Re-running threshold_experiment_and_classify to get the metrics object
# Parameters are taken from the last detoxify experiment in cell LXGIlffRKNDW
df_processed, metrics_detoxify, dfs_detoxify = threshold_experiment_and_classify(
    df_anotado.copy(), # Use a copy to avoid modifying the original df_anotado prematurely
    label_cols=['toxico1', 'toxico2', 'toxico3'],
    tox_score='detoxify_toxicity',
    do_plots=False # Set to False to avoid regenerating plots, as the user only asked to display metrics
)

print(f"=== MELHOR THRESHOLD (Detoxify): {metrics_detoxify['best_threshold']:.2f} ===")
print(f"Acurácia Global: {metrics_detoxify['final']['accuracy']:.4f}")
print("-" * 40)

print("\n### 1. Tabela Resumo: Métricas Finais por Classe")
print(dfs_detoxify['final_por_classe'].to_string(index=False))
print("-" * 40)

print("\n### 2. Curvas de Métricas vs Threshold (Macro)")
print("F1 Macro:", [f"{v:.2f}" for v in metrics_detoxify['threshold_curves']['macro']['f1']])
print("Precision Macro:", [f"{v:.2f}" for v in metrics_detoxify['threshold_curves']['macro']['precision']])
print("Recall Macro:", [f"{v:.2f}" for v in metrics_detoxify['threshold_curves']['macro']['recall']])
print("-" * 40)

print("\n### 3. Curvas de Métricas vs Threshold (Por Classe - Classe 0)")
print("F1 Classe 0:", [f"{v:.2f}" for v in metrics_detoxify['threshold_curves']['per_class']['0']['f1']])
print("Precision Classe 0:", [f"{v:.2f}" for v in metrics_detoxify['threshold_curves']['per_class']['0']['precision']])
print("Recall Classe 0:", [f"{v:.2f}" for v in metrics_detoxify['threshold_curves']['per_class']['0']['recall']])
print("-" * 40)

print("\n### 4. Curvas de Métricas vs Threshold (Por Classe - Classe 1)")
print("F1 Classe 1:", [f"{v:.2f}" for v in metrics_detoxify['threshold_curves']['per_class']['1']['f1']])
print("Precision Classe 1:", [f"{v:.2f}" for v in metrics_detoxify['threshold_curves']['per_class']['1']['precision']])
print("Recall Classe 1:", [f"{v:.2f}" for v in metrics_detoxify['threshold_curves']['per_class']['1']['recall']])
print("-" * 40)

print("\n### 5. Matrizes de Confusão Absolutas por Threshold (Lista de Arrays)")
for i, cm in enumerate(metrics_detoxify['threshold_curves']['confusion_matrices']['absolute']):
    print(f"Threshold {metrics_detoxify['thresholds'][i]:.1f}:\n{np.array(cm)}")
print("-" * 40)

print("\n### 6. Matrizes de Confusão Normalizadas por Threshold (Lista de Arrays)")
for i, cm in enumerate(metrics_detoxify['thresholds']):
    print(f"Threshold {metrics_detoxify['thresholds'][i]:.1f}:\n{np.array(metrics_detoxify['threshold_curves']['confusion_matrices']['normalized'][i])}")
print("-" * 40)

=== MELHOR THRESHOLD (Detoxify): 0.30 ===
Acurácia Global: 0.7545
----------------------------------------

### 1. Tabela Resumo: Métricas Finais por Classe
 classe  support  precision   recall       f1
      0       55   0.507937 0.581818 0.542373
      1      165   0.853503 0.812121 0.832298
----------------------------------------

### 2. Curvas de Métricas vs Threshold (Macro)
F1 Macro: ['0.54', '0.67', '0.69', '0.65', '0.62', '0.58', '0.51', '0.43', '0.33', '0.20']
Precision Macro: ['0.60', '0.69', '0.68', '0.65', '0.64', '0.64', '0.64', '0.62', '0.64', '0.12']
Recall Macro: ['0.55', '0.66', '0.70', '0.69', '0.69', '0.68', '0.65', '0.60', '0.56', '0.50']
----------------------------------------

### 3. Curvas de Métricas vs Threshold (Por Classe - Classe 0)
F1 Classe 0: ['0.24', '0.49', '0.54', '0.53', '0.52', '0.52', '0.49', '0.45', '0.43', '0.40']
Precision Classe 0: ['0.43', '0.56', '0.51', '0.44', '0.39', '0.37', '0.33', '0.30', '0.28', '0.25']
Recall Classe 0: ['0.16', '0.44'

/tmp/ipython-input-3827924404.py:109: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  counts_df = df.groupby(['toxicity_range', 'toxicity_classification']).size().unstack(fill_value=0)


Organiza as métricas em um dataframe (visualmente mais organizado)

In [36]:
# --- 1. Execução do Experimento ---
df_processado, metricas_dict, dfs_extras = threshold_experiment_and_classify(
    df_anotado.copy(),
    label_cols=['toxico1', 'toxico2', 'toxico3'],
    tox_score='detoxify_toxicity',
    do_plots=False
)

# --- 2. Extração de Métricas ---
c0 = metricas_dict['final']['per_class']['0']
c1 = metricas_dict['final']['per_class']['1']

# --- 3. Cálculo de Macros ---
macro_prec = (c0['precision'] + c1['precision']) / 2
macro_rec  = (c0['recall']    + c1['recall'])    / 2
macro_f1_val = (c0['f1']      + c1['f1'])        / 2

# --- 4. Criação do DataFrame (Nomes Exatos) ---
df_tabela_metricas = pd.DataFrame([{
    'model/test':      'Detoxify',
    'macro_precision': macro_prec,
    'macro_recall':    macro_rec,
    'macro_f1':        macro_f1_val,
    'class0precision': c0['precision'],
    'class0recall':    c0['recall'],
    'class0f1':        c0['f1'],
    'class1precision': c1['precision'],
    'class1recall':    c1['recall'],
    'class1f1':        c1['f1']
}])

# --- 5. Exibição ---
print(f"=== Threshold: {metricas_dict['best_threshold']} | Acurácia: {metricas_dict['final']['accuracy']:.4f} ===")
display(df_tabela_metricas)

=== Threshold: 0.30000000000000004 | Acurácia: 0.7545 ===


/tmp/ipython-input-3827924404.py:109: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  counts_df = df.groupby(['toxicity_range', 'toxicity_classification']).size().unstack(fill_value=0)


,model/test,macro_precision,macro_recall,macro_f1,class0precision,class0recall,class0f1,class1precision,class1recall,class1f1
0,Detoxify,0.68072,0.69697,0.687336,0.507937,0.581818,0.542373,0.853503,0.812121,0.832298


Gera a Matriz de Confusão

# Detoxify

In [11]:
import numpy as np
import matplotlib.pyplot as plt

def plot_toxicity_cdf(df, column):
    # 1. Pega a coluna de toxicidade de todos os vídeos
    toxicity_scores = df[column].dropna().values

    # 2. Ordena os valores em ordem crescente (essencial para CDF)
    sorted_scores = np.sort(toxicity_scores)

    # 3. Cria os valores acumulados (proporção)
    cdf = np.arange(1, len(sorted_scores) + 1) / len(sorted_scores)

    # 4. Cria o gráfico
    plt.figure(figsize=(6, 3))
    plt.plot(sorted_scores, cdf, color='darkred', lw=2, label='CDF')

    # 5. Adiciona linha vertical no ponto 0.7
    plt.axvline(x=0.7, color='gray', linestyle='--', lw=1)
    plt.text(0.7, 0.05, '0.7', ha='center', fontsize=8)

    # 6. Calcula e mostra % de vídeos com toxicidade > 0.7
    perc_above = np.sum(toxicity_scores > 0.7) / len(toxicity_scores) * 100
    plt.text(0.72, 0.6, f'{perc_above:.1f}% > 0.7', color='black', fontsize=9)

    # 7. Formatações finais
    plt.xlabel(column)
    plt.ylabel("Proporção Acumulada (CDF)")
    plt.grid(False)
    plt.ylim(0, 1.01)
    plt.xlim(0, 1)
    plt.tight_layout()
    plt.show()


In [12]:
import pandas as pd
# from google.colab import drive

# drive.mount('/content/drive')
folder_path = ''

def experimento(df_merged, algoritmo, tox_cols, tox_score, min_val, max_val, n_bins, label1, label2, label3):
  print("Ground truth por classe -----------------------------------------")
  df_merged = plot_ground_truth_por_classe(df_merged, tox_score, label_cols=[label1, label2, label3])
  print("Anotações por classe por faixa -----------------------------------------")
  result = tabela_anotacao_por_faixa(df_merged, tox_score, min_val, max_val, n_bins)
  df_final, metrics, dfs = threshold_experiment_and_classify(
      df_merged,
      label_cols=[label1, label2, label3],
      tox_score=tox_score,
      do_plots=False
  )

  # Métricas finais por classe
  final_cls0 = metrics['final']['per_class']['0']
  final_cls1 = metrics['final']['per_class']['1']

  print("Melhor theta:", metrics['best_threshold'])

  print("\n=== F1 por classe (wide) ===")
  print(dfs['f1_por_classe_wide'].to_string(index=False))

  print("\n=== Precision por classe (wide) ===")
  print(dfs['precision_por_classe_wide'].to_string(index=False))

  print("\n=== Recall por classe (wide) ===")
  print(dfs['recall_por_classe_wide'].to_string(index=False))

  print("\n=== Matriz de confusão (absoluta) ===")
  df_cm_abs_long = dfs['confusao_abs_long']

  # Chamar a função de plotagem
  thresholds_list = np.linspace(0.1, 1.0, 10)
  plot_all_confusion_matrices_in_grid(
    df_cm_long=df_cm_abs_long,
    thresholds=thresholds_list,
    title='Matrizes de Confusão Absoluta por Threshold'
  )

  print("\n=== Métricas finais por classe ===")
  print(dfs['final_por_classe'].to_string(index=False))

  df_merged.to_csv(folder_path + f'classificados_{algoritmo}_apos_tunning.csv')

## Detoxify Misoginia

In [13]:
import pandas as pd
from detoxify import Detoxify
from tqdm import tqdm

# Certifique-se de que seu DataFrame possui a coluna 'text'
# Exemplo: df_videos = pd.DataFrame({'text': ["Você é incrível", "Seu idiota nojento!"]})

# Carrega o modelo adequado para múltiplos idiomas
model = Detoxify('multilingual')

# Função para aplicar a análise linha a linha
def analyze_toxicity_detoxify(texts):
    results = []
    for text in tqdm(texts, desc="Analisando toxicidade"):
        try:
            scores = model.predict(text)
        except Exception:
            # Se ocorrer erro, retorna pontuações nulas
            scores = {k: None for k in model.predict("teste").keys()}
        results.append(scores)
    return results

Downloading: "https://github.com/unitaryai/detoxify/releases/download/v0.4-alpha/multilingual_debiased-0b549669.ckpt" to /root/.cache/torch/hub/checkpoints/multilingual_debiased-0b549669.ckpt
100%|██████████| 1.04G/1.04G [00:20<00:00, 54.8MB/s]
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [14]:
from detoxify import Detoxify
import pandas as pd

def classificar_com_detoxify(df_para_classificar: pd.DataFrame, df_scores_detoxify: pd.DataFrame) -> pd.DataFrame:
    RESP_COL = "text"
    DETOX_PREFIX = "detoxify_"

    # ===== Selecionar colunas de interesse nos scores salvos =====
    detox_cols = [col for col in df_scores_detoxify.columns if col.startswith(DETOX_PREFIX)]
    cols_para_merge = [RESP_COL] + detox_cols

    # ===== Remover duplicatas nos textos de scores salvos =====
    df_scores_unicos = df_scores_detoxify[cols_para_merge].drop_duplicates(subset=[RESP_COL])

    # ===== Fazer merge para adicionar os scores existentes =====
    df_classificado = df_para_classificar.merge(
        df_scores_unicos,
        on=RESP_COL,
        how='left'
    )

    # ===== Identificar textos ainda não classificados =====
    df_nao_classificados = df_classificado[
        df_classificado[f"{DETOX_PREFIX}toxicity"].isnull() & df_classificado[RESP_COL].notnull()
    ]

    print(f"Textos novos para classificar: {len(df_nao_classificados)}")

    # ===== Classificar os textos restantes =====
    if not df_nao_classificados.empty:
        model = Detoxify('multilingual')
        textos_novos = df_nao_classificados[RESP_COL].tolist()
        resultados = model.predict(textos_novos)

        # ===== Atribuir os resultados aos índices corretos =====
        idxs_para_preencher = df_classificado[
            df_classificado[f"{DETOX_PREFIX}toxicity"].isnull()
        ].index

        for key in resultados:
            col_name = f"{DETOX_PREFIX}{key}"
            df_classificado.loc[idxs_para_preencher, col_name] = resultados[key]

    return df_classificado


In [16]:
import pandas as pd

tox_score = 'toxicity'

print("Lendo anotações -----------------------------------------")

import pandas as pd
from google.colab import drive

drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/red_pill_analises/data_base/dados_anotados/'

df_ground_truth = pd.read_csv(folder_path + 'anotacoes_completas.csv', encoding='utf-8')
df_ground_truth.rename(columns={'texto_original': 'text'}, inplace=True)

print("Lendo scores previamente calculados ----------------------")
df_scores_detoxify = pd.read_csv('/content/drive/MyDrive/red_pill_analises/data_base/resultados/' + 'videos_com_scores_de_toxicidade.csv')

tox_cols = ['toxicity', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']
algoritmo = 'videos_com_detoxify'

print(df_ground_truth.columns, df_scores_detoxify.columns)

print("Classsificando textos -------------------------------------")
df_anotado = classificar_com_detoxify(df_ground_truth, df_scores_detoxify)

Lendo anotações -----------------------------------------
Mounted at /content/drive
Lendo scores previamente calculados ----------------------
Index(['text', 'misoginia1', 'misoginia2', 'misoginia3', 'toxico1', 'toxico2',
       'toxico3', 'categoria1', 'categoria2', 'categoria3'],
      dtype='object') Index(['Unnamed: 0', 'published_at', 'category_id', 'tags', 'view_count',
       'like_count', 'comment_count', 'duration', 'definition', 'caption',
       'licensed_content', 'privacy_status', 'license', 'embeddable',
       'public_stats_viewable', 'is_made_for_kids', 'thumbnail_url',
       'default_audio_language', 'default_language', 'actual_start_time',
       'scheduled_start_time', 'actual_end_time', 'scheduled_end_time',
       'concurrent_viewers', 'active_live_chat_id', 'recording_date',
       'topicCategories', 'processing_status', 'parts_total',
       'parts_processed', 'time_left_ms', 'processing_failure_reason',
       'transcription', 'id_canal_anonimizado', 'id_video_

In [ ]:
df_anotado.to_csv('scores_detoxify_atualizado.csv')

In [ ]:
df_anotado = pd.read_csv('scores_detoxify_atualizado.csv')

tox_cols = ['detoxify_toxicity']

df_anotado['is_offensive'] = (df_anotado['detoxify_toxicity'] > 0.7).astype(int)
df_anotado.head()

print(df_anotado.columns)

experimento(df_anotado, algoritmo, tox_cols, 'detoxify_toxicity', min_val=0, max_val=1, n_bins=10, label1='misoginia1', label2='misoginia2', label3='misoginia3')

df_anotado.to_csv('anotado_detoxify_misoginia.csv')

## Detoxify para Toxicidade

In [ ]:
tox_cols = ['detoxify_toxicity', 'detoxify_severe_toxicity', 'detoxify_obscene', 'detoxify_identity_attack', 'detoxify_insult', 'detoxify_threat']

df_anotado['is_offensive'] = (df_anotado[tox_cols] >= 0.7).any(axis=1).astype(int)
df_anotado.head()

print(df_anotado.columns)

experimento(df_anotado, algoritmo, tox_cols, 'detoxify_toxicity', min_val=0, max_val=1, n_bins=10, label1='toxico1', label2='toxico2', label3='toxico3')

df_anotado.to_csv('anotado_detoxify_misoginia.csv')

# GuardRail

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

def score_toxiguardrail_prompt_fixo(
    tokenizer,
    device,
    toxiGuardrail,
    df: pd.DataFrame,
    response_col: str,
    prompt: str,
    batch_size: int = 32,
    max_length: int = 512,
) -> pd.Series:
    responses = df[response_col].fillna("").astype(str).tolist()
    prompts = [prompt] * len(responses)

    scores = []
    for i in range(0, len(responses), batch_size):
        p_batch = prompts[i:i+batch_size]
        r_batch = responses[i:i+batch_size]

        tokens = tokenizer(
            p_batch,
            r_batch,
            truncation=True,
            max_length=max_length,
            padding=True,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors="pt",
        )
        tokens = {k: v.to(device) for k, v in tokens.items()}

        with torch.no_grad():
            out = toxiGuardrail(**tokens)
            logits = out.logits

        if logits.ndim == 2 and logits.shape[1] == 1:
            batch_scores = logits.squeeze(-1).detach().cpu().tolist()
        else:
            probs = torch.softmax(logits, dim=-1).detach().cpu()
            id2label = toxiGuardrail.config.id2label
            toxic_idx = None
            for k, v in id2label.items():
                name = str(v).lower()
                if any(x in name for x in ["toxic", "unsafe", "harm", "tóxic", "viol", "negat"]):
                    toxic_idx = int(k)
                    break
            if toxic_idx is None:
                toxic_idx = 1 if probs.shape[1] > 1 else 0
            batch_scores = probs[:, toxic_idx].tolist()

        scores.extend(batch_scores)

    return pd.Series(scores, index=df.index, name="guardrail_score")

def classificar_com_guardrail(df_para_classificar: pd.DataFrame, df_scores_guardrail: pd.DataFrame) -> pd.DataFrame:
    RESP_COL = "text"
    SCORE_COL = "guardrail_score"

    # ===== Selecionar colunas de interesse =====
    cols_para_merge = [RESP_COL, SCORE_COL]

    # ===== Remover duplicatas dos scores =====
    df_scores_unicos = df_scores_guardrail[cols_para_merge].drop_duplicates(subset=[RESP_COL])

    # ===== Fazer merge para adicionar os scores existentes =====
    df_classificado = df_para_classificar.merge(
        df_scores_unicos,
        on=RESP_COL,
        how='left'
    )

    # ===== Identificar textos ainda não classificados =====
    df_nao_classificados = df_classificado[
        df_classificado[SCORE_COL].isnull() & df_classificado[RESP_COL].notnull()
    ]

    print(f"Textos novos para classificar com Guardrail: {len(df_nao_classificados)}")

    # ===== Classificar os textos restantes =====
    if not df_nao_classificados.empty:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        CKPT = "nicholasKluge/ToxiGuardrailPT"
        tokenizer = AutoTokenizer.from_pretrained(CKPT)
        toxiGuardrail = AutoModelForSequenceClassification.from_pretrained(CKPT)
        toxiGuardrail.eval().to(device)

        prompt = "Classifique textos que são misóginos ou não"

        novos_scores = score_toxiguardrail_prompt_fixo(
            tokenizer,
            device,
            toxiGuardrail,
            df_nao_classificados,
            response_col=RESP_COL,
            prompt=prompt,
            batch_size=32,
            max_length=512
        )

        # ===== Atribuir os resultados aos índices corretos =====
        idxs_para_preencher = df_classificado[
            df_classificado[SCORE_COL].isnull()
        ].index

        df_classificado.loc[idxs_para_preencher, SCORE_COL] = novos_scores

    return df_classificado


## ToxiGuard para Misoginia

In [ ]:
df_anotado = classificar_com_guardrail(df_ground_truth, df_scores_detoxify)

In [ ]:
df_anotado1 = df_anotado
experimento(df_anotado1, algoritmo, tox_cols, 'guardrail_score', min_val=-6, max_val=6, n_bins=10, label1='misoginia1', label2='misoginia2', label3='misoginia3')

## ToxiGuard para Toxicidade

In [ ]:
tox_score = 'guardrail_score'
tox_cols = ['guardrail_score']

df_anotado['is_offensive'] = (df_anotado['guardrail_score'] <= -0.5).astype(int)
df_anotado.head()

experimento(df_anotado, algoritmo, tox_cols, 'guardrail_score', min_val=-6, max_val=6, n_bins=10, label1='toxico1', label2='toxico2', label3='toxico3')

df_anotado.to_csv('anotado_toxiguard_toxicidade.csv')

# Doug Trajano

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

def classificar_com_dougtrajano(df_para_classificar: pd.DataFrame, df_scores_doug: pd.DataFrame) -> pd.DataFrame:
    RESP_COL = "text"  # coluna com os textos
    PREFIX = "doug_tox_"

    # ===== Selecionar colunas de interesse nos scores salvos =====
    cols_para_merge = [col for col in df_scores_doug.columns if col.startswith(PREFIX)]
    cols_para_merge = [RESP_COL] + cols_para_merge

    # ===== Remover duplicatas nos textos de scores salvos =====
    df_scores_unicos = df_scores_doug[cols_para_merge].drop_duplicates(subset=[RESP_COL])

    # ===== Fazer merge para adicionar os scores existentes =====
    df_classificado = df_para_classificar.merge(
        df_scores_unicos,
        on=RESP_COL,
        how="left"
    )

    # ===== Identificar textos ainda não classificados =====
    alguma_col_score = [col for col in df_classificado.columns if col.startswith(f"{PREFIX}") and col.endswith("_score")]
    if alguma_col_score:
        df_nao_classificados = df_classificado[df_classificado[alguma_col_score[0]].isnull() & df_classificado[RESP_COL].notnull()]
    else:
        df_nao_classificados = df_classificado[df_classificado[RESP_COL].notnull()]

    print(f"Textos novos para classificar: {len(df_nao_classificados)}")

    # ===== Classificar os textos restantes =====
    if not df_nao_classificados.empty:
        CKPT = "dougtrajano/toxicity-type-detection"
        tokenizer = AutoTokenizer.from_pretrained(CKPT)
        model = AutoModelForSequenceClassification.from_pretrained(CKPT)
        model.eval()

        # Detectar tipo de problema automaticamente (ou force se necessário)
        # model.config.problem_type = "multi_label_classification"

        def analyze_toxicity(texts: pd.Series, model, tokenizer):
            from torch.nn.functional import sigmoid, softmax

            max_pos = getattr(model.config, "max_position_embeddings", 512)
            window = max_pos - 2  # para CLS e SEP
            stride = 64
            microbatch_size = 16
            problem_type = getattr(model.config, "problem_type", None)

            id2label = model.config.id2label
            num_labels = model.config.num_labels
            labels = [id2label[i] for i in range(num_labels)]

            PAD_ID = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else 0

            def _chunks_by_tokens(text: str):
                enc_full = tokenizer(text, add_special_tokens=False, return_attention_mask=False)
                ids = enc_full["input_ids"]
                if len(ids) <= window:
                    return [tokenizer.build_inputs_with_special_tokens(ids)]
                chunks = []
                start = 0
                while start < len(ids):
                    end = min(start + window, len(ids))
                    chunk_ids = tokenizer.build_inputs_with_special_tokens(ids[start:end])
                    chunks.append(chunk_ids)
                    if end == len(ids):
                        break
                    start = max(0, end - stride)
                return chunks

            def _pad_batch(batch):
                max_len_batch = max(len(ids) for ids in batch)
                input_ids = []
                attn_masks = []
                for ids in batch:
                    pad_len = max_len_batch - len(ids)
                    input_ids.append(ids + [PAD_ID] * pad_len)
                    attn_masks.append([1] * len(ids) + [0] * pad_len)
                input_ids = torch.tensor(input_ids, dtype=torch.long, device=device)
                attn_masks = torch.tensor(attn_masks, dtype=torch.long, device=device)
                return input_ids, attn_masks

            def _forward_on_chunks(chunks):
                logits_list = []
                for i in range(0, len(chunks), microbatch_size):
                    batch = chunks[i:i + microbatch_size]
                    input_ids, attention_mask = _pad_batch(batch)
                    with torch.no_grad():
                        out = model(input_ids=input_ids, attention_mask=attention_mask)
                    logits_list.append(out.logits.detach().cpu())
                return torch.cat(logits_list, dim=0)

            rows = []
            for text in texts.fillna("").astype(str):
                chunks = _chunks_by_tokens(text)
                if not chunks:
                    score_vec = [0.0] * num_labels
                    bin_vec = [0] * num_labels
                else:
                    logits = _forward_on_chunks(chunks)
                    if problem_type == "multi_label_classification":
                        probs = sigmoid(logits)
                        agg = probs.max(dim=0).values
                        score_vec = agg.tolist()
                        bin_vec = [int(s >= 0.5) for s in score_vec]
                    else:
                        probs = softmax(logits, dim=-1)
                        agg = probs.mean(dim=0)
                        score_vec = agg.tolist()
                        top1 = int(torch.argmax(agg).item())
                        bin_vec = [1 if i == top1 else 0 for i in range(num_labels)]

                row = {}
                for i, label in enumerate(labels):
                    row[f"{PREFIX}{label}_score"] = float(score_vec[i])
                    row[f"{PREFIX}{label}_pred"] = int(bin_vec[i])
                rows.append(row)

            return pd.DataFrame(rows)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)

        novos_scores = analyze_toxicity(df_nao_classificados[RESP_COL], model, tokenizer)
        novos_scores[RESP_COL] = df_nao_classificados[RESP_COL].values

        # ===== Recombinar os resultados =====
        df_classificado = df_classificado.drop(columns=[col for col in df_classificado.columns if col.startswith(PREFIX)], errors="ignore")
        df_classificado = df_classificado.merge(novos_scores, on=RESP_COL, how="left")

    return df_classificado


## OLID BR Misoginia

In [ ]:
tox_score = 'tox_sexism_score'

print("Classificando sampled_df ---------------")
algoritmo = 'videos_com_dougtrajano'
df_anotado = classificar_com_dougtrajano(df_ground_truth, df_scores_detoxify)

In [ ]:
print("Identificando ofensivos ---------------")
print(len(df_anotado))
print(df_anotado.columns)
df_anotado['is_offensive'] = df_anotado['doug_tox_sexism_pred']

print("Identificou os textos ofensivos")
print(df_anotado.head())

#tox_cols = ['toxicity', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']
tox_cols = ['doug_tox_insult_score', 'doug_tox_lgbtqphobia_score',
       'doug_tox_other_lifestyle_score', 'doug_tox_physical_aspects_score',
       'doug_tox_profanity_obscene_score', 'doug_tox_racism_score', 'doug_tox_sexism_score',
       'doug_tox_xenophobia_score']
algoritmo = 'videos_com_dougtrajano'

df_olid = df_anotado
print(df_anotado.head())

In [ ]:
print(df_anotado.columns)

In [ ]:
from collections import defaultdict

# 1. Renomeia colunas duplicadas com sufixo _dupX para que fiquem únicas temporariamente
def rename_duplicate_columns(df):
    seen = defaultdict(int)
    new_cols = []
    for col in df.columns:
        count = seen[col]
        if count:
            new_cols.append(f"{col}_dup{count}")
        else:
            new_cols.append(col)
        seen[col] += 1
    df.columns = new_cols
    return df

df_anotado = rename_duplicate_columns(df_anotado)

# 2. Encontra todas as colunas que eram 'tox_sexism_score'
sexism_cols = [col for col in df_anotado.columns if col.startswith('tox_sexism_score')]

if len(sexism_cols) > 1:
    # 3. Calcula número de NaN para cada uma
    null_counts = [df_anotado[col].isnull().sum() for col in sexism_cols]

    # 4. Identifica a que tem mais NaN e remove
    col_to_drop = sexism_cols[null_counts.index(max(null_counts))]
    df_anotado.drop(columns=col_to_drop, inplace=True)

    # 5. Renomeia a que sobrou (a boa) para 'tox_sexism_score'
    for col in sexism_cols:
        if col != col_to_drop:
            df_anotado.rename(columns={col: 'tox_sexism_score'}, inplace=True)
            break


In [ ]:
df_anotado['doug_tox_sexism_score']

In [ ]:
print("Iniciando experimento -------------------")
experimento(df_anotado, algoritmo=algoritmo, tox_cols=tox_cols, tox_score='doug_tox_sexism_score', min_val=0, max_val=1, n_bins=10, label1="misoginia1", label2="misoginia2", label3="misoginia3")

In [ ]:
print(df_anotado.head())

## OLID BR com Toxicidade

In [ ]:
df_anotado['is_offensive'] = (df_anotado[tox_cols] >= 0.7).any(axis=1).astype(int)
df_anotado.head()

print("Iniciando experimento -------------------")
experimento(df_anotado, algoritmo=algoritmo, tox_cols=tox_cols, tox_score='doug_tox_sexism_score', min_val=0, max_val=1, n_bins=10, label1="toxico1", label2="toxico2", label3="toxico3")

In [ ]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/red_pill_analises/data_base/dados_anotados/'
tox_score = 'toxicity'

print("Lendo anotações -----------------------------------------")
df_ground_truth = pd.read_csv(folder_path + 'anotacoes_completas.csv', encoding='utf-8')

## Avaliação Misoginia

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score
from statsmodels.stats.inter_rater import fleiss_kappa

# Preencher NaNs com 0
df_ground_truth[['misoginia1', 'misoginia2', 'misoginia3']] = df_ground_truth[['misoginia1', 'misoginia2', 'misoginia3']].fillna(0)

# ===== Cálculo do Kappa de Cohen entre pares =====
kappa_1_2 = cohen_kappa_score(df_ground_truth['misoginia1'], df_ground_truth['misoginia2'])
kappa_1_3 = cohen_kappa_score(df_ground_truth['misoginia1'], df_ground_truth['misoginia3'])
kappa_2_3 = cohen_kappa_score(df_ground_truth['misoginia2'], df_ground_truth['misoginia3'])

print("Kappa (Misoginia 1 vs 2):", kappa_1_2)
print("Kappa (Misoginia 1 vs 3):", kappa_1_3)
print("Kappa (Misoginia 2 vs 3):", kappa_2_3)

# ===== Cálculo do número de textos com consenso total =====
df_ground_truth['consenso_total'] = (
    (df_ground_truth['misoginia1'] == df_ground_truth['misoginia2']) &
    (df_ground_truth['misoginia1'] == df_ground_truth['misoginia3'])
)
quantidade_consenso = df_ground_truth['consenso_total'].sum()
print("Textos com consenso total:", quantidade_consenso)

# ===== Fleiss' Kappa =====
# Supondo que os rótulos possíveis sejam 0 e 1
labels = [0, 1]
matriz_contagem = []

for _, row in df_ground_truth[['misoginia1', 'misoginia2', 'misoginia3']].iterrows():
    contagem = [list(row).count(label) for label in labels]
    matriz_contagem.append(contagem)

matriz_contagem = np.array(matriz_contagem)
fleiss = fleiss_kappa(matriz_contagem, method='fleiss')
print("Fleiss' Kappa:", fleiss)


In [ ]:
df_ground_truth['consenso_total'] = (
    (df_ground_truth['misoginia1'] == df_ground_truth['misoginia2']) &
    (df_ground_truth['misoginia1'] == df_ground_truth['misoginia3'])
)

quantidade_consenso = df_ground_truth['consenso_total'].sum()
print("Textos com consenso total:", quantidade_consenso)

## Avaliação Misoginia

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score
from statsmodels.stats.inter_rater import fleiss_kappa

# Preencher NaNs com 0
df_ground_truth[['toxico1', 'toxico2', 'toxico3']] = df_ground_truth[['toxico1', 'toxico2', 'toxico3']].fillna(0)

# ===== Cálculo do Kappa de Cohen entre pares =====
kappa_1_2 = cohen_kappa_score(df_ground_truth['toxico1'], df_ground_truth['toxico2'])
kappa_1_3 = cohen_kappa_score(df_ground_truth['toxico1'], df_ground_truth['toxico3'])
kappa_2_3 = cohen_kappa_score(df_ground_truth['toxico2'], df_ground_truth['toxico3'])

print("Kappa (toxico 1 vs 2):", kappa_1_2)
print("Kappa (toxico 1 vs 3):", kappa_1_3)
print("Kappa (toxico 2 vs 3):", kappa_2_3)

# ===== Cálculo do número de textos com consenso total =====
df_ground_truth['consenso_total'] = (
    (df_ground_truth['toxico1'] == df_ground_truth['toxico2']) &
    (df_ground_truth['toxico1'] == df_ground_truth['toxico3'])
)
quantidade_consenso = df_ground_truth['consenso_total'].sum()
print("Textos com consenso total:", quantidade_consenso)

# ===== Fleiss' Kappa =====
# Supondo que os rótulos possíveis sejam 0 e 1
labels = [0, 1]
matriz_contagem = []

for _, row in df_ground_truth[['toxico1', 'toxico2', 'toxico3']].iterrows():
    contagem = [list(row).count(label) for label in labels]
    matriz_contagem.append(contagem)

matriz_contagem = np.array(matriz_contagem)
fleiss = fleiss_kappa(matriz_contagem, method='fleiss')
print("Fleiss' Kappa:", fleiss)


In [ ]:
df_ground_truth['consenso_total'] = (
    (df_ground_truth['toxico1'] == df_ground_truth['toxico2']) &
    (df_ground_truth['toxico1'] == df_ground_truth['toxico3'])
)

quantidade_consenso = df_ground_truth['consenso_total'].sum()
print("Textos com consenso total:", quantidade_consenso)

In [ ]:
import pandas as pd
from scipy.stats import spearmanr

# Exemplo: carregando DataFrame
# dg_ground_truth = pd.read_csv("seus_dados.csv")

# Calcula correlação
corr, p_value = spearmanr(dg_ground_truth['is_toxico'], dg_ground_truth['is_misogino'])

print(f"Correlação de Spearman: {corr:.4f}")
print(f"p-valor: {p_value:.4f}")